# Dask jobqueue example

## What is Dask jobqueue? (https://jobqueue.dask.org/)

* deploys Dask workers on typical HPC job queueing systems

## Monte-Carlo estimate with multiple Dask batch job workers
We define a Dask jobqueue cluster with Dask workers that each have 4 CPUs and 24 GB of memory.

In [1]:
import dask, dask.distributed
import dask_jobqueue

dask.config.set({'jobqueue.pbs.walltime': None});  # NQS workaround

In [2]:
cluster = dask_jobqueue.PBSCluster(

    # Dask workers
    cores=4, memory='24GB',
    processes=1, # workers per job

    # PBS job script
    queue='cltestque', name='dask-worker',
    resource_spec=('elapstim_req=00:45:00,'
                   'cpunum_job=4,memsz_job=24gb'),
    local_directory='/scratch',
    interface='ib0',
)

client = dask.distributed.Client(cluster)
cluster.scale(jobs=1)

In [3]:
client

Client Scheduler: tcp://192.168.31.13:40380 Dashboard: http://192.168.31.13:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


### What is a jobqueue cluster?
The above is all we need to specify to run the computation on compute node Dask workers. 
Let's have a look at what's happening in the background.

In [4]:
!qstat

RequestID       ReqName  UserName Queue     Pri STT S   Memory      CPU   Elapse R H M Jobs
--------------- -------- -------- -------- ---- --- - -------- -------- -------- - - - ----
362911.nesh-bat dask-wor smomw260 cltestqu    0 QUE -    0.00B     0.00        0 Y Y Y    1 


In [5]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -q cltestque
#PBS -l elapstim_req=00:45:00,cpunum_job=4,memsz_job=24gb
JOB_ID=${PBS_JOBID%%.*}

/sfs/fs6/home-geomar/smomw260/miniconda3/envs/dask-minimal-20191218/bin/python -m distributed.cli.dask_worker tcp://192.168.31.13:40380 --nthreads 4 --memory-limit 24.00GB --name name --nanny --death-timeout 60 --local-directory /scratch --interface ib0



### Let's scale up the cluster

In [7]:
cluster.scale(jobs=8)

In [14]:
!qstat

RequestID       ReqName  UserName Queue     Pri STT S   Memory      CPU   Elapse R H M Jobs
--------------- -------- -------- -------- ---- --- - -------- -------- -------- - - - ----
362911.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  155.09M     7.81      128 Y Y Y    1 
362916.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  156.57M     5.41       27 Y Y Y    1 
362917.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  164.08M     3.52       28 Y Y Y    1 
362918.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  165.37M     3.32       28 Y Y Y    1 
362919.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  166.83M     5.07       27 Y Y Y    1 
362920.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  163.40M     5.03       27 Y Y Y    1 
362921.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  164.27M     4.30       26 Y Y Y    1 
362922.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  158.27M     4.12       26 Y Y Y    1 


In [15]:
client

Client Scheduler: tcp://192.168.31.13:40380 Dashboard: http://192.168.31.13:8787/status,Cluster Workers: 8 Cores: 32 Memory: 192.00 GB


### From here everything is the same as with LocalCluster

In [16]:
import numpy, dask.array

def calculate_pi(size_in_bytes, number_of_chunks):
    
    """Calculate pi using a Monte Carlo method."""
    
    array_shape = (int(size_in_bytes / 8 / 2), 2) # tuple
    chunk_size = (int(array_shape[0] / number_of_chunks), 2) # tuple
    
    # 2D random positions array using dask.array
    xy = dask.array.random.uniform(
        low=0.0, high=1.0, size=array_shape,
        # specify chunk size, i.e. task number
        chunks=chunk_size )
  
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1 # boolean

    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size
    
    # start Dask calculation
    pi = pi.compute()

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    # display(xy)
    
    return pi

### Let's calculate again...

In [17]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1416782848
   pi error: 8.56312102071044e-05

CPU times: user 224 ms, sys: 22.5 ms, total: 247 ms
Wall time: 1.81 s


In [18]:
%time pi = calculate_pi(size_in_bytes=100_000_000_000, number_of_chunks=250) # 100 GB


from 100.0 GB randomly chosen positions
   pi estimate: 3.14161525952
   pi error: 2.2605930206864855e-05

CPU times: user 1.26 s, sys: 176 ms, total: 1.44 s
Wall time: 17.2 s


In [19]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=1000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141586118144
   pi error: 6.535445792987815e-06

CPU times: user 12.1 s, sys: 978 ms, total: 13.1 s
Wall time: 2min 53s


### We can scale up the cluster whenever needed

In [20]:
cluster.scale(jobs=16)

In [28]:
!qstat

RequestID       ReqName  UserName Queue     Pri STT S   Memory      CPU   Elapse R H M Jobs
--------------- -------- -------- -------- ---- --- - -------- -------- -------- - - - ----
362911.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  271.55M   646.88      408 Y Y Y    1 
362916.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  269.12M   652.89      307 Y Y Y    1 
362917.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  278.24M   647.01      308 Y Y Y    1 
362918.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  275.72M   660.38      308 Y Y Y    1 
362919.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  277.25M   648.40      307 Y Y Y    1 
362920.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  275.46M   660.81      307 Y Y Y    1 
362921.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  275.61M   652.89      306 Y Y Y    1 
362922.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  269.23M   652.02      306 Y Y Y    1 
362959.nesh-bat dask-wor smomw260 cltestqu    0 RUN -  165.58M     3.95 

In [29]:
client

Client Scheduler: tcp://192.168.31.13:40380 Dashboard: http://192.168.31.13:8787/status,Cluster Workers: 16 Cores: 64 Memory: 384.00 GB


### Let's calculate again...

In [30]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=1000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141599593408
   pi error: 6.939818206763704e-06

CPU times: user 9.27 s, sys: 652 ms, total: 9.92 s
Wall time: 1min 33s


In [ ]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10000) # 10 TB

### Note, we could also adaptively scale the jobqueue cluster!

Dask jobqueue is able to scale total worker number based on problem size. You can also specify a target duration.

In [ ]:
cluster.adapt(
    minimum=2, maximum=20,
    target_duration="60s"
)

In [ ]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB

In [ ]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=1000) # 1 TB